In [6]:
from pydantic import BaseModel
from openai import AzureOpenAI
import os
import json
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import re
from azure.cosmos import CosmosClient, exceptions, PartitionKey
from dotenv import load_dotenv
import os

In [7]:
# Load environment variables from .env file
load_dotenv()

def read_json_files_from_blob(folder_path):
    # Retrieve the connection string from the environment variables
    connection_string = os.getenv('STORAGE_CONNECTION_STRING')

    # Ensure the connection string is not None
    if connection_string is None:
        raise ValueError("The connection string environment variable is not set.")

    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get the container client
    container_client = blob_service_client.get_container_client("data")

    # List all blobs in the specified folder
    blob_list = container_client.list_blobs(name_starts_with=folder_path)

    # Filter out JSON files and read their contents
    for blob in blob_list:
        if blob.name.endswith('.json'):
            blob_client = container_client.get_blob_client(blob.name)
            blob_data = blob_client.download_blob().readall()
            data = json.loads(blob_data)
            return data 

In [9]:
houseloan = read_json_files_from_blob("houseloan")

In [10]:
houseloan

{'handwritten': False,
 'pages': [{'page_number': 1,
   'width': 8.2639,
   'height': 11.6806,
   'unit': 'inch',
   'lines': [{'text': 'Contoso Bank - House Loan Terms and Conditions',
     'polygon': [0.9866,
      1.0104,
      4.4787,
      1.0113,
      4.4787,
      1.1715,
      0.9866,
      1.1707],
     'words': [{'content': 'Contoso', 'confidence': 0.995},
      {'content': 'Bank', 'confidence': 0.992},
      {'content': '-', 'confidence': 0.995},
      {'content': 'House', 'confidence': 0.996},
      {'content': 'Loan', 'confidence': 0.989},
      {'content': 'Terms', 'confidence': 0.995},
      {'content': 'and', 'confidence': 0.998},
      {'content': 'Conditions', 'confidence': 0.993}]},
    {'text': '1. Introduction',
     'polygon': [0.9941,
      1.3297,
      2.0273,
      1.3298,
      2.0273,
      1.4808,
      0.9941,
      1.4806],
     'words': [{'content': '1.', 'confidence': 0.995},
      {'content': 'Introduction', 'confidence': 0.992}]},
    {'text': 'These

In [ ]:
def clean_json_data(json_data):
    # Extract relevant text content from the JSON
    content = []

    # Extract text from paragraphs
    paragraphs = json_data.get("paragraphs", [])
    for paragraph in paragraphs:
        content.append(paragraph.get("text", "").strip())

    # Extract text from pages and lines
    pages = json_data.get("pages", [])
    for page in pages:
        for line in page.get("lines", []):
            content.append(line.get("text", "").strip())

    # Join all text content into a single string with spaces between components
    plain_text_content = " ".join(content)

    # Extract Customer ID using regex
    pattern = r"Customer ID:\s*(\d+)"
    match = re.search(pattern, plain_text_content)
    customer_id = match.group(1) if match else None
    return plain_text_content, customer_id

# Clean the JSON data and extract Customer ID
loanagreement_structured, customer_id = clean_json_data(loanagreement)